## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-04-15-05-41 +0000,48hills,Drama Masks: Feel good this plot is not,https://48hills.org/2026/02/drama-masks-feel-g...
1,2026-02-04-15-02-00 +0000,wsj,Opinion | Nail Dartmouth’s Five Theses to Coll...,https://www.wsj.com/opinion/nail-dartmouths-fi...
2,2026-02-04-15-01-00 +0000,wsj,Opinion | Alternatives to Shaking Down Social ...,https://www.wsj.com/opinion/alternatives-to-sh...
3,2026-02-04-15-01-00 +0000,wsj,The Dell Scion Who Wants to Shore Up the Texas...,https://www.wsj.com/business/energy-oil/the-de...
4,2026-02-04-15-00-14 +0000,nyt,Minneapolis Police Chief Fears Long-Term Damag...,https://www.nytimes.com/2026/02/04/us/brian-oh...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2440/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
150,trump,37
352,epstein,22
40,new,21
397,house,20
151,ice,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
232,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...,116
250,2026-02-03-22-56-00 +0000,wsj,The Republican-led House passed funding legisl...,https://www.wsj.com/politics/policy/government...,105
270,2026-02-03-22-17-00 +0000,wsj,President Trump shared a rendering Tuesday of ...,https://www.wsj.com/politics/policy/trump-whit...,102
217,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...,89
241,2026-02-03-23-21-07 +0000,bbc,Trump hails White House talks with 'terrific' ...,https://www.bbc.com/news/articles/c4g01vlwqp5o...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
232,116,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...
5,67,2026-02-04-15-00-00 +0000,wsj,Republicans and Democrats have less than two w...,https://www.wsj.com/politics/policy/congress-e...
333,66,2026-02-03-19-41-36 +0000,nypost,Jeffrey Epstein asked woman to buy Snow White ...,https://nypost.com/2026/02/03/business/jeffrey...
74,54,2026-02-04-12-35-16 +0000,nyt,Savannah Guthrie to Skip Olympics as Search fo...,https://www.nytimes.com/2026/02/03/us/savannah...
217,52,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...
219,51,2026-02-04-00-54-23 +0000,cbc,French police raid X offices as they investiga...,https://www.cbc.ca/news/world/french-police-ra...
330,49,2026-02-03-19-49-51 +0000,nypost,Nelson Peltz accuses Bob Iger of rigging Disne...,https://nypost.com/2026/02/03/business/nelson-...
20,48,2026-02-04-14-35-16 +0000,startribune,"Minnesota school districts, teachers union sue...",https://www.startribune.com/minnesota-school-d...
10,42,2026-02-04-14-52-00 +0000,wsj,"America’s private sector added 22,000 jobs las...",https://www.wsj.com/economy/jobs/adp-numbers-s...
320,40,2026-02-03-20-11-18 +0000,nypost,Ukrainian husband weeps beside childhood sweet...,https://nypost.com/2026/02/03/world-news/ukrai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
